In [1]:
import paho.mqtt.client as mqtt
import ast
import numpy as np
from collections import deque
from sklearn.neighbors import KNeighborsClassifier


4. Modify the MQTT example to do the following:
- The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the
beginning again.
- The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the
last 5 records before r, and compare the classification result with the Outcome label in r.
- Repeat this for 1000 times, and report the number of correct classifications.

In [2]:

# Store received records
data_buffer = deque(maxlen=6)  # Keeps the last 6 records (5 for training, 1 for testing)
correct_predictions = 0
iterations = 0

# MQTT Callback functions
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("diabetes/data")

def on_message(client, userdata, msg):
    global correct_predictions, iterations
    
    # Convert message payload to dictionary
    record = ast.literal_eval(msg.payload.decode())
    print(f"Received {iterations+1}: {record}")
    
    data_buffer.append(record)
    
    if len(data_buffer) == 6:  # Ensure we have at least 6 records
        train_data = list(data_buffer)[:-1]
        test_record = list(data_buffer)[-1]
        
        # Prepare training data
        X_train = np.array([[r['Glucose'], r['BloodPressure'], r['Insulin']] for r in train_data])
        y_train = np.array([r['Outcome'] for r in train_data])
        
        # Prepare test record
        X_test = np.array([[test_record['Glucose'], test_record['BloodPressure'], test_record['Insulin']]])
        y_actual = test_record['Outcome']
        
        # Train 3NN Classifier
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(X_train, y_train)
        
        # Make Prediction
        y_pred = knn.predict(X_test)[0]
        print(f"Predicted Outcome: {y_pred}, Actual Outcome: {y_actual}")
        
        if y_pred == y_actual:
            correct_predictions += 1
        
        iterations += 1
        
        if iterations >= 1000:
            print(f"Total Correct Classifications: {correct_predictions} out of 1000")
            client.disconnect()

# Setup MQTT Client
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# Start listening for messages
mqttc.loop_forever()


C:\Users\abjar\AppData\Local\Temp\ipykernel_1580\1477035485.py:50: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqttc = mqtt.Client()


Connected with result code 0
Received 1: {'Glucose': 0.7437185929648241, 'BloodPressure': 0.5901639344262295, 'Insulin': 0.0, 'Outcome': 1}
Received 1: {'Glucose': 0.4271356783919598, 'BloodPressure': 0.5409836065573771, 'Insulin': 0.0, 'Outcome': 0}
Received 1: {'Glucose': 0.9195979899497488, 'BloodPressure': 0.5245901639344263, 'Insulin': 0.0, 'Outcome': 1}
Received 1: {'Glucose': 0.4472361809045226, 'BloodPressure': 0.5409836065573771, 'Insulin': 0.1111111111111111, 'Outcome': 0}
Received 1: {'Glucose': 0.6884422110552764, 'BloodPressure': 0.3278688524590164, 'Insulin': 0.1985815602836879, 'Outcome': 1}
Received 1: {'Glucose': 0.5829145728643216, 'BloodPressure': 0.6065573770491803, 'Insulin': 0.0, 'Outcome': 0}
Predicted Outcome: 0, Actual Outcome: 0
Received 2: {'Glucose': 0.3919597989949749, 'BloodPressure': 0.4098360655737705, 'Insulin': 0.1040189125295508, 'Outcome': 1}
Predicted Outcome: 0, Actual Outcome: 1
Received 3: {'Glucose': 0.577889447236181, 'BloodPressure': 0.0, 'Ins

<MQTTErrorCode.MQTT_ERR_CONN_LOST: 7>